# Action recognition

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import pickle
import os

## Dataloader 

In [6]:
DATA_PATH = "data/"

In [54]:
def get_data( name, data_path = DATA_PATH):
# label = 0: roll_droite, 1: roll_gauche, 2: salut_droite, 3: salut_gauche (determiné apar l'ordre des pkl dans le dossier data)
    master_df = pd.DataFrame(columns = ["frame","left_shoulder", "right_shoulder", "left_elbow", "right_elbow", "label"])
    label = 0
    for pickle_file in os.listdir(DATA_PATH):
        if pickle_file.endswith(name+".pkl"):
            temp_df = pd.read_pickle(DATA_PATH + pickle_file)
            temp_df["label"] = label
            label += 1
            master_df = pd.concat([master_df, temp_df], axis = 0)
    return master_df

In [55]:
master_df = pd.concat([get_data("gui"), get_data("val")], axis = 0)

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df, window_size):
        self.data = []
        self.labels = []
        for name, group in df.groupby('series_id'):
            label = group['label'].iloc[0] # Assuming all time steps in the same series have the same label
            series = group['value'].values
            for i in range(len(series)-window_size+1):
                window = series[i:i+window_size]
                self.data.append(window)
                self.labels.append(label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx]), torch.tensor(self.labels[idx])

In [ ]:
batch_size = 32
window_size = 5
df = pd.read_csv('data.csv')
dataset = TimeSeriesDataset(df, window_size)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

## Model

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
input_size = 1 # Size of each time step in the input window
hidden_size = 64 # Number of features in the hidden state of the LSTM
num_layers = 2 # Number of LSTM layers
num_classes = 4 # Number of output classes (i.e. number of possible labels)
model = LSTMModel(input_size, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.unsqueeze(-1) # Add extra dimension for input_size
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(dataset)//batch_size, loss.item()))